In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('Solarize_Light2')

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import seaborn as sns

from folium.plugins import HeatMapWithTime

import datetime as dt

import missingno as msno

import random

import os
print(os.listdir("/Users/yc00027/Documents/air-quality-madrid/"))

import warnings
warnings.filterwarnings("ignore")

['.DS_Store', 'csvs_per_year', 'stations.csv', 'madrid.h5', '.ipynb_checkpoints']


In [3]:
# Import full dataframe

df_full = pd.read_csv('full_madrid_air_quality.csv').set_index('date').sort_index()
df_full.index = pd.to_datetime(df_full.index)

particles_of_interest = ['NO_2', 'O_3', 'PM10', 'SO_2']

df_full['all_particles'] = df_full[particles_of_interest].sum(axis=1)

# Select data for the time period, particles and stations of interest

first_date = '2012-01-01 00:00:00'
last_date = '2018-04-30 23:00:00'

stations_of_interest = [28079008, 28079018, 28079024]

df_temp = df_full.loc[first_date:last_date][particles_of_interest + ['station', 'all_particles']].sort_index()
#df_temp = df_temp.loc[df_temp['station'].isin(stations_of_interest)]
df_temp['quarter'] = df_temp.index.quarter
df_temp['dayofyear'] = df_temp.index.dayofyear
df_temp['dayofmonth'] = df_temp.index.day
df_temp['weekyear'] = df_temp.index.weekofyear
df_temp['weekday'] = df_temp.index.weekday
df_temp['hour'] = df_temp.index.hour
df_temp['year'] = df_temp.index.year
df_temp['month'] = df_temp.index.month
df_temp.fillna(0, inplace=True)

print(f'The time period is from {first_date} to {last_date}')
print('')
print(f'The particles of interest are: {particles_of_interest}')
print('')
#print(f'The stations of interest are: {stations_of_interest}')
#print('')
print(f'One particle DF shape: {df_temp.shape}')

df_temp.head()

The time period is from 2012-01-01 00:00:00 to 2018-04-30 23:00:00

The particles of interest are: ['NO_2', 'O_3', 'PM10', 'SO_2']

One particle DF shape: (2164032, 14)


,NO_2,O_3,PM10,SO_2,station,all_particles,quarter,dayofyear,dayofmonth,weekyear,weekday,hour,year,month
date,,,,,,,,,,,,,,
2012-01-01,0.0,0.0,0.0,0.0,28079001,0.0,1,1,1,52,6,0,2012,1
2012-01-01,0.0,0.0,0.0,0.0,28079022,0.0,1,1,1,52,6,0,2012,1
2012-01-01,0.0,0.0,0.0,0.0,28079019,0.0,1,1,1,52,6,0,2012,1
2012-01-01,0.0,0.0,0.0,0.0,28079021,0.0,1,1,1,52,6,0,2012,1
2012-01-01,0.0,0.0,0.0,0.0,28079012,0.0,1,1,1,52,6,0,2012,1


In [4]:
df_temp[particles_of_interest + ['all_particles']].describe()

,NO_2,O_3,PM10,SO_2,all_particles
count,2.164032e+06,2.164032e+06,2.164032e+06,2.164032e+06,2.164032e+06
mean,2.340833e+01,1.717874e+01,6.057906e+00,1.484539e+00,4.812952e+01
std,2.996491e+01,3.051362e+01,1.338139e+01,3.547513e+00,4.904260e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.200000e+01
75%,3.800000e+01,2.500000e+01,7.000000e+00,1.000000e+00,8.400000e+01
max,4.240000e+02,2.360000e+02,4.510000e+02,1.040000e+02,5.500000e+02


In [5]:
# Extract station locations into a separate dataframe

df_stations = pd.read_hdf('madrid.h5', key='master')
df_stations['name_id'] = df_stations['name'] +' ID: ' + df_stations['id'].astype(str)

print(df_stations.shape)
df_stations.head()

(24, 7)


,id,name,address,lon,lat,elevation,name_id
0,28079004,Pza. de España,Plaza de España,-3.712247,40.423853,635,Pza. de España ID: 28079004
1,28079008,Escuelas Aguirre,Entre C/ Alcalá y C/ O’ Donell,-3.682319,40.421564,670,Escuelas Aguirre ID: 28079008
2,28079011,Avda. Ramón y Cajal,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,-3.677356,40.451475,708,Avda. Ramón y Cajal ID: 28079011
3,28079016,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,-3.639233,40.440047,693,Arturo Soria ID: 28079016
4,28079017,Villaverde,C/. Juan Peñalver,-3.713322,40.347139,604,Villaverde ID: 28079017


In [7]:
# Map the locations of the stations

locations  = df_stations[['lat', 'lon']]
locationlist = locations.values.tolist()

popup = df_stations[['name_id']]

import folium
base_map = folium.Map(location=[40.44, -3.69], zoom_start=12) 

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(base_map)

base_map.save('basemap.html')
base_map

In [8]:
# Merge geo locations from locations dataframe with zomato dataframe

df_map = df_temp.merge(df_stations[['id', 'name', 'lat', 'lon']], left_on = ['station'], right_on = ['id'], how = 'left')
df_map.index = df_temp.index
df_map.drop(columns=['id'], inplace=True)
print(df_map.shape)
df_map.head()

(2164032, 17)


,NO_2,O_3,PM10,SO_2,station,all_particles,quarter,dayofyear,dayofmonth,weekyear,weekday,hour,year,month,name,lat,lon
date,,,,,,,,,,,,,,,,,
2012-01-01,0.0,0.0,0.0,0.0,28079001,0.0,1,1,1,52,6,0,2012,1,NaN,NaN,NaN
2012-01-01,0.0,0.0,0.0,0.0,28079022,0.0,1,1,1,52,6,0,2012,1,NaN,NaN,NaN
2012-01-01,0.0,0.0,0.0,0.0,28079019,0.0,1,1,1,52,6,0,2012,1,NaN,NaN,NaN
2012-01-01,0.0,0.0,0.0,0.0,28079021,0.0,1,1,1,52,6,0,2012,1,NaN,NaN,NaN
2012-01-01,0.0,0.0,0.0,0.0,28079012,0.0,1,1,1,52,6,0,2012,1,NaN,NaN,NaN


In [9]:
data_to_plot = df_map.loc['2017-01-01':]
heat_df = []
particle_of_interest = 'all_particles'
for i, (index, row) in enumerate(data_to_plot.iterrows()):
    if i % 1000 == 0:
        print(index)
    
    station = row['name']
    lat = row['lat']
    lon = row['lon']
    _data = [{
        'datetime': index,
        particle_of_interest: 1,
        'station': station,
        'lat': lat,
        'lon': lon,        
    } for i in range(int(row[particle_of_interest] / 10))]
    heat_df.extend(_data)

heat_df = pd.DataFrame(heat_df)
print(heat_df.shape)
heat_df.head()

2017-01-01 00:00:00
2017-01-02 01:00:00
2017-01-03 03:00:00
2017-01-04 04:00:00
2017-01-05 06:00:00
2017-01-06 08:00:00
2017-01-07 09:00:00
2017-01-08 11:00:00
2017-01-09 13:00:00
2017-01-10 14:00:00
2017-01-11 16:00:00
2017-01-12 18:00:00
2017-01-13 19:00:00
2017-01-14 21:00:00
2017-01-15 22:00:00
2017-01-17 00:00:00
2017-01-18 02:00:00
2017-01-19 03:00:00
2017-01-20 05:00:00
2017-01-21 07:00:00
2017-01-22 08:00:00
2017-01-23 10:00:00
2017-01-24 12:00:00
2017-01-25 13:00:00
2017-01-26 15:00:00
2017-01-27 17:00:00
2017-01-28 18:00:00
2017-01-29 20:00:00
2017-01-30 21:00:00
2017-01-31 23:00:00
2017-02-02 01:00:00
2017-02-03 02:00:00
2017-02-04 04:00:00
2017-02-05 06:00:00
2017-02-06 07:00:00
2017-02-07 09:00:00
2017-02-08 11:00:00
2017-02-09 12:00:00
2017-02-10 14:00:00
2017-02-11 16:00:00
2017-02-12 17:00:00
2017-02-13 19:00:00
2017-02-14 20:00:00
2017-02-15 22:00:00
2017-02-17 00:00:00
2017-02-18 01:00:00
2017-02-19 03:00:00
2017-02-20 05:00:00
2017-02-21 06:00:00
2017-02-22 08:00:00


2018-03-16 02:00:00
2018-03-17 04:00:00
2018-03-18 05:00:00
2018-03-19 07:00:00
2018-03-20 09:00:00
2018-03-21 10:00:00
2018-03-22 12:00:00
2018-03-23 13:00:00
2018-03-24 15:00:00
2018-03-25 17:00:00
2018-03-26 18:00:00
2018-03-27 20:00:00
2018-03-28 22:00:00
2018-03-29 23:00:00
2018-03-31 01:00:00
2018-04-01 03:00:00
2018-04-02 04:00:00
2018-04-03 06:00:00
2018-04-04 08:00:00
2018-04-05 09:00:00
2018-04-06 11:00:00
2018-04-07 12:00:00
2018-04-08 14:00:00
2018-04-09 16:00:00
2018-04-10 17:00:00
2018-04-11 19:00:00
2018-04-12 21:00:00
2018-04-13 22:00:00
2018-04-15 00:00:00
2018-04-16 02:00:00
2018-04-17 03:00:00
2018-04-18 05:00:00
2018-04-19 06:00:00
2018-04-20 08:00:00
2018-04-21 10:00:00
2018-04-22 11:00:00
2018-04-23 13:00:00
2018-04-24 15:00:00
2018-04-25 16:00:00
2018-04-26 18:00:00
2018-04-27 20:00:00
2018-04-28 21:00:00
2018-04-29 23:00:00
(2108857, 5)


,datetime,all_particles,station,lat,lon
0,2017-01-01,1,Ensanche de Vallecas,40.372933,-3.612117
1,2017-01-01,1,Ensanche de Vallecas,40.372933,-3.612117
2,2017-01-01,1,Ensanche de Vallecas,40.372933,-3.612117
3,2017-01-01,1,Ensanche de Vallecas,40.372933,-3.612117
4,2017-01-01,1,Ensanche de Vallecas,40.372933,-3.612117


In [10]:
# Visualise levels of pollution in stations of interest throughout the days of the month

basemap2 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap2)

df_day_list = []
for day in heat_df['datetime'].dt.day.sort_values().unique():
    df_day_list.append(heat_df.loc[heat_df['datetime'].dt.day == day, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist()) 
    
gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}

hm = HeatMapWithTime(df_day_list, radius=100, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True, gradient=gradient, auto_play=True)
hm.add_to(basemap2)
basemap2.save('daysofmonthmap.html')
basemap2

In [11]:
# Visualise levels of pollution in stations of interest throughout the hours of the day

basemap3 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap3)

df_hour_list = []
for hour in heat_df['datetime'].dt.hour.sort_values().unique():
    df_hour_list.append(heat_df.loc[heat_df['datetime'].dt.hour == hour, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist())   

gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}

hm = HeatMapWithTime(df_hour_list, radius=100, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True, gradient=gradient, auto_play=True)
hm.add_to(basemap3)
basemap3.save('hoursmap.html')
basemap3

In [12]:
# Visualise levels of pollution in stations of interest throughout the days of the year

basemap4 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap4)

df_dayofyear_list = []
for dayofyear in heat_df['datetime'].dt.dayofyear.sort_values().unique():
    df_dayofyear_list.append(heat_df.loc[heat_df['datetime'].dt.dayofyear == dayofyear, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist())   

hm = HeatMapWithTime(df_dayofyear_list, radius=100, min_opacity=0.5, max_opacity=0.8, \
                             use_local_extrema=True, gradient=gradient, auto_play=True)

hm.add_to(basemap4)
basemap4.save('daysofyearmap.html')
basemap4

In [13]:
# Visualise levels of pollution in stations of interest throughout the weeks of the year

basemap5 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap5)

df_weekofyear_list = []
for weekofyear in heat_df['datetime'].dt.weekofyear.sort_values().unique():
    df_weekofyear_list.append(heat_df.loc[heat_df['datetime'].dt.weekofyear == weekofyear, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist())   

hm = HeatMapWithTime(df_weekofyear_list, radius=100, min_opacity=0.5, max_opacity=0.8, \
                             use_local_extrema=True, gradient=gradient, auto_play=True)

hm.add_to(basemap5)
basemap5.save('weeksofyearmap.html')
basemap5

In [14]:
# Visualise levels of pollution in stations of interest throughout the days of the week

basemap6 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap6)

df_weekday_list = []
for weekday in heat_df['datetime'].dt.weekday.sort_values().unique():
    df_weekday_list.append(heat_df.loc[heat_df['datetime'].dt.weekday == weekday, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist())   

hm = HeatMapWithTime(df_weekday_list, radius=100, min_opacity=0.5, max_opacity=0.8, \
                             use_local_extrema=True, gradient=gradient, auto_play=True)

hm.add_to(basemap6)
basemap6.save('weekdaysmap.html')
basemap6

In [15]:
# Visualise levels of pollution in stations of interest throughout the months

basemap7 = folium.Map(location=[40.44, -3.69], zoom_start=12)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=popup.iloc[point,0]).add_to(basemap7)

df_month_list = []
for month in heat_df['datetime'].dt.month.sort_values().unique():
    df_month_list.append(heat_df.loc[heat_df['datetime'].dt.month == month, \
    ['lat', 'lon', particle_of_interest]].groupby(['lat', 'lon']).sum().reset_index().values.tolist())   

hm = HeatMapWithTime(df_month_list, radius=100, min_opacity=0.5, max_opacity=0.8, \
                             use_local_extrema=True, gradient=gradient, auto_play=True)

hm.add_to(basemap7)
basemap7.save('monthsmap.html')
basemap7